This notebook combines datasets on GEIH Metro areas for checking trends across years.

In [2]:
import pandas as pd
import re

import sys
sys.path.insert(0, '../utils')
from settings import *

In [35]:
def clean_name(text):
    return (re.sub('[^a-z ]','', text.lower()).replace(' ', '_')
            .replace('area_metropolitana_de_', '')
            .replace('area_metropolitana_del_', ''))

In [36]:
wash18 = pd.read_csv(data_dir + '20200830_dataset.csv').drop_duplicates('id')
grid_in_metro = pd.read_csv(data_dir + 'grids_in_metro_areas.csv')
metro19 = pd.read_csv(data_dir + '20200831_GEIH_Metro_Areas.csv')
metro20 = pd.read_csv(data_dir + '20200908_GEIH_Metro_Areas_2020.csv')
metro_name = pd.read_csv(data_dir + 'metro_areas_id_name.csv')

In [40]:
metro_name['a_mtro'] = metro_name['a_mtro'].apply(clean_name)
metro_name = metro_name.rename(columns = {'OBJECTID': 'metro_id'})

## actual

In [37]:
spanish = {
    'd_hogares': 'population',
    'd_c_acuedu': 'hh_no_water_supply',
    'd_c_alcant': 'hh_no_sewage',
    'd_c_sanita': 'hh_no_toilet',
}

df1 = (pd.merge(grid_in_metro, wash18[['id'] + list(spanish.keys())], how = 'left', on = 'id')
      .rename(columns = spanish))
print(df1.shape)
df2 = df1.groupby('metro_id').agg('sum').reset_index()
for indicator in indicators:
    df2[indicator] = 100*df2[indicator.replace('perc_', '')] / df2['population']

metro18 = df2

(26542, 10)


In [38]:
spanish = {
    'OBJECTID': 'metro_id',
    'personas': 'population',
    'c_acueduct': 'hh_no_water_supply',
    'c_alcantar': 'hh_no_sewage',
    'c_sanitari': 'hh_no_toilet',
    'mc_acueduc': 'perc_hh_no_water_supply',
    'mc_alcanta': 'perc_hh_no_sewage',
    'mc_sanitar': 'perc_hh_no_toilet',
}

metro19 = metro19.rename(columns = spanish)
metro20 = metro20.rename(columns = spanish)

In [42]:
cols = ['metro_id', 'year'] + indicators

metro18['year'] = 2018
metro19['year'] = 2019
metro20['year'] = 2020

df3 = pd.concat([
    metro18[cols],
    metro19[cols],
    metro20[cols],   
], axis = 0)

In [49]:
df4 = pd.merge(metro_name, df3, how = 'left', on = 'metro_id')
df5 = df4.set_index(['metro_id', 'a_mtro', 'year']).stack().reset_index()
df5.columns = ['metro_id', 'a_mtro', 'year', 'indicator', 'value']

In [ ]:
df5.to_csv(data_dir + 'metro_trends.csv', index = False)

## predicted

In [50]:
pred_metro19 = pd.read_csv(data_dir + 'metro_area_predictions.csv')
pred_metro20 = pd.read_csv(data_dir + 'metro_area_predictions_2020.csv')

In [57]:
metro18[cols].shape

(23, 5)

In [ ]:
pred_metro19

In [60]:
pred_metro19.rename(columns = rnm)[cols].shape

(23, 8)

In [62]:
rnm = {
    'pred_perc_hh_no_water_supply': 'perc_hh_no_water_supply', 
    'pred_perc_hh_no_toilet': 'perc_hh_no_toilet', 
    'pred_perc_hh_no_sewage': 'perc_hh_no_sewage'
}

pred_metro19['year'] = 2019
pred_metro20['year'] = 2020

cols2 = ['metro_id', 'year'] + list(rnm.keys())
df6 = pd.concat([
    metro18[cols],
    pred_metro19[cols2].rename(columns = rnm),
    pred_metro20[cols2].rename(columns = rnm)
], axis = 0)

df7 = pd.merge(metro_name, df6, how = 'left', on = 'metro_id')
df8 = df7.set_index(['metro_id', 'a_mtro', 'year']).stack().reset_index()
df8.columns = ['metro_id', 'a_mtro', 'year', 'indicator', 'value']

In [65]:
df5['val_type'] = 'actual'
df8['val_type'] = 'pred'
df9 = pd.concat([df5, df8], axis = 0)

## output

In [67]:
df9.to_csv(data_dir + 'metro_trends.csv', index = False)

In [63]:
df8.shape

(207, 5)

In [64]:
df5.shape

(207, 5)

In [51]:
pred_metro19.columns

Index(['Unnamed: 0', 'metro_id', 'population_x', 'pred_hh_no_water_supply',
       'pred_hh_no_toilet', 'pred_hh_no_sewage',
       'pred_perc_hh_no_water_supply', 'pred_perc_hh_no_toilet',
       'pred_perc_hh_no_sewage', 'geometry', 'population_y',
       'hh_no_water_supply', 'hh_no_sewage', 'hh_no_toilet',
       'perc_hh_no_water_supply', 'perc_hh_no_sewage', 'perc_hh_no_toilet'],
      dtype='object')

In [52]:
pred_metro20.columns

Index(['Unnamed: 0', 'metro_id', 'population_x', 'pred_hh_no_water_supply',
       'pred_hh_no_toilet', 'pred_hh_no_sewage',
       'pred_perc_hh_no_water_supply', 'pred_perc_hh_no_toilet',
       'pred_perc_hh_no_sewage', 'geometry', 'population_y',
       'hh_no_water_supply', 'hh_no_sewage', 'hh_no_toilet',
       'perc_hh_no_water_supply', 'perc_hh_no_sewage', 'perc_hh_no_toilet'],
      dtype='object')

In [ ]:
# make 2018 data with metro id by joining
# append existing 2019,2020 data
# append all